In [9]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").getOrCreate()
from pyspark.sql.functions import col

In [7]:
# Azure storage access info
blob_account_name = "azureopendatastorage"
blob_container_name = "nyctlc"
blob_relative_path = "yellow"
blob_sas_token = r""

# Allow SPARK to read from Blob remotely
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set(
  'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
  blob_sas_token)
print('Remote blob path: ' + wasbs_path)

# SPARK read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)
print('Register the DataFrame as a SQL temporary view: source')
df.createOrReplaceTempView('source')

# Display top 10 rows
print('Displaying top 10 rows: ')
display(spark.sql('SELECT * FROM source LIMIT 10'))

Remote blob path: wasbs://nyctlc@azureopendatastorage.blob.core.windows.net/yellow
Register the DataFrame as a SQL temporary view: source
Displaying top 10 rows: 


DataFrame[vendorID: string, tpepPickupDateTime: timestamp, tpepDropoffDateTime: timestamp, passengerCount: int, tripDistance: double, puLocationId: string, doLocationId: string, startLon: double, startLat: double, endLon: double, endLat: double, rateCodeId: int, storeAndFwdFlag: string, paymentType: string, fareAmount: double, extra: double, mtaTax: double, improvementSurcharge: string, tipAmount: double, tollsAmount: double, totalAmount: double, puYear: int, puMonth: int]

In [10]:
df_filtered = df.filter((col("puYear") >= 2015) & (col("puYear") <= 2018))

In [12]:
print('Displaying the first 100 rows: ')
df_filtered.show(100)

Displaying the first 100 rows: 
+--------+-------------------+-------------------+--------------+------------+------------+------------+------------------+------------------+------------------+------------------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|vendorID| tpepPickupDateTime|tpepDropoffDateTime|passengerCount|tripDistance|puLocationId|doLocationId|          startLon|          startLat|            endLon|            endLat|rateCodeId|storeAndFwdFlag|paymentType|fareAmount|extra|mtaTax|improvementSurcharge|tipAmount|tollsAmount|totalAmount|puYear|puMonth|
+--------+-------------------+-------------------+--------------+------------+------------+------------+------------------+------------------+------------------+------------------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+-----------+------+-------+
|       2|2015-02-28 23:32